MODULOS

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

Creacion Data Frames

In [2]:
df_ALUCarrera = pd.read_csv('Alu_Carrera.csv', sep=';')
df_AluMateria = pd.read_csv('Alu_Materia.csv', sep=';')
df_Carreras = pd.read_csv('Carreras.csv', sep=';')
df_Finales = pd.read_csv('Finales.csv', sep=';')
df_Materias = pd.read_csv('Materias.csv', sep=';')

LIMPIEZA

Data Time a los DF

In [3]:
#Arreglo el problema de que los datos de tipo tiempo estan como si fueran tipo: objeto en vez de tipo: DataTime
df_ALUCarrera[['Fecha_Ingreso','Fecha_Nac']] = df_ALUCarrera[['Fecha_Ingreso','Fecha_Nac']].apply(pd.to_datetime)
df_AluMateria[['Fecha_Inscrip','Fecha_1p','Fecha_2p','Fecha_TP','Fecha_Recup']] = df_AluMateria[['Fecha_Inscrip','Fecha_1p','Fecha_2p','Fecha_TP','Fecha_Recup']].apply(pd.to_datetime)
df_Finales[['Fecha_Examen','Fecha_Inscrip_Examen']] = df_Finales[['Fecha_Examen','Fecha_Inscrip_Examen']].apply(pd.to_datetime)

Nota Media Finales x Año

In [4]:
df_Finales_Pre = df_Finales[df_Finales['Nota'] < 11]
#Creo la columa Año para poder tenerlo y asi usarlo en el groupby de mas adelante
df_Finales_Pre['Año'] = pd.DatetimeIndex(df_Finales_Pre['Fecha_Examen']).year

<ipython-input-4-1d88b074142a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Finales_Pre['Año'] = pd.DatetimeIndex(df_Finales_Pre['Fecha_Examen']).year


In [5]:
#Uno el DF Carreras con el de finales para tener los nombres de las carreras
df_Finales_Pre = pd.merge(df_Finales_Pre,
                            df_Carreras[['Id_Carrera','Nombre_Corto']],
                            how='left',
                            on=['Id_Carrera'])
df_Finales_Pre

,Id_Alumno,Id_Carrera,Id_Materia,Semetre_Cursada,Año_Cursada,Fecha_Examen,Nota,Fecha_Inscrip_Examen,Año,Nombre_Corto
0,0xD2B32B6CE5A79C6AF97CACC9E4BAE8A4,2,68,1,2001,2001-07-27,9.0,NaT,2001,MARKETING
1,0xD2B32B6CE5A79C6AF97CACC9E4BAE8A4,2,69,1,2001,2001-07-24,10.0,NaT,2001,MARKETING
2,0xD2B32B6CE5A79C6AF97CACC9E4BAE8A4,2,70,1,2001,2001-07-19,7.0,NaT,2001,MARKETING
3,0xD2B32B6CE5A79C6AF97CACC9E4BAE8A4,2,71,2,2001,2001-12-07,5.0,NaT,2001,MARKETING
4,0xD2B32B6CE5A79C6AF97CACC9E4BAE8A4,2,72,2,2001,2002-02-26,8.0,NaT,2002,MARKETING
...,...,...,...,...,...,...,...,...,...,...
123542,0x24C9300567A882456A355E68B2980753,5,253,1,2004,2004-07-30,5.0,NaT,2004,DISEÑO GRÁFICO
123543,0x24C9300567A882456A355E68B2980753,5,256,2,2003,2003-12-15,6.0,NaT,2003,DISEÑO GRÁFICO
123544,0x24C9300567A882456A355E68B2980753,5,256,2,2003,2003-12-15,6.0,NaT,2003,DISEÑO GRÁFICO
123545,0x24C9300567A882456A355E68B2980753,5,259,1,2001,2001-07-26,9.0,NaT,2001,DISEÑO GRÁFICO


In [6]:
#Creo un df que tenga el groupby de la nota media de cada carrera por año
Nota_Media = df_Finales_Pre.groupby(['Nombre_Corto','Año'])['Nota'].mean().reset_index(name='Cantidad')
Nota_Media

,Nombre_Corto,Año,Cantidad
0,ADMINISTRACIÓN DE EMPRESAS,2001,6.208651
1,ADMINISTRACIÓN DE EMPRESAS,2002,6.276158
2,ADMINISTRACIÓN DE EMPRESAS,2003,5.622691
3,ADMINISTRACIÓN DE EMPRESAS,2004,6.021762
4,ADMINISTRACIÓN DE EMPRESAS,2005,5.937449
...,...,...,...
310,VIDEOJUEGOS,2015,3.428571
311,VIDEOJUEGOS,2016,7.500000
312,VIDEOJUEGOS,2017,3.600000
313,VIDEOJUEGOS,2018,5.947368


# Eliminacion de Carreras 11,60,18,39,59,40,41,12

In [7]:
#Averiguo como se llaman las carreras
listaDeCarreras = [11,60,18,39,59,40,41,16,12]
df_Carreras[df_Carreras['Id_Carrera'].isin(listaDeCarreras)]

,Id_Carrera,Nombre_Largo,Nombre_Corto,Abreviatura,Cuatrimestres
2,12,ANALISTA EN GESTION DE VENTAS,VENTAS,VTA,4
6,18,TECNICO SUPERIOR EN LOGISTICA,LOGÍSTICA,LOG,6
8,41,AGENTE DE TRANSPORTE ADUANERO Y DESPACHANTE DE...,AGENTE DE TRANSP. ADUANERO Y DESP. DE ADUANA,ATA,4
12,59,TÉCNICO SUPERIOR EN SERVICIOS DE PAISAJISMO,SERVICIOS DE PAISAJISMO,SERPAI,2
13,39,ADMINISTRADOR DE REDES,ADMINISTRADOR DE REDES,AR,4
14,16,TECNICO SUPERIOR EN GESTION JURIDICA,GESTIÓN JURíDICA,GJ,6
17,60,TÉC. SUP. EN DES. DE SIMULACIONES VIRTUALES Y ...,VIDEOJUEGOS,SVIRT,6
18,40,TÉCNICO EN SERVICIOS AGROPECUARIOS,TÉCNICO EN SERVICIOS AGROPECUARIOS,TSAGR,4
19,11,ASISTENTE GERENCIAL TRILINGÜE,TRILINGÜE,TRI,4


In [8]:
#Averiguo los alumnos que estan en cada carrera
df_ALUCarrera[df_ALUCarrera['Id_Carrera']==12]

,Id_Alumno,Id_Carrera,Semestre_Ingreso,Año_Ingreso,Id_Ctro_Ext,Id_Modalidad,Fecha_Ingreso,Activo,Sexo,Fecha_Nac
6,0x2F0ABFF4F547FA62CBCD41A047F618AA,12,2,2001,CO,P,2001-07-23,S,M,1963-04-26
1107,0xD23D90DC739F5A9A59EDF40E032201F8,12,2,2001,CO,P,2001-08-31,S,M,1981-06-16


In [9]:
#Elimino todo registro de las carreras que tienen datos ignorables
df_Carreras = df_Carreras[~df_Carreras['Id_Carrera'].isin(listaDeCarreras)]
df_Finales = df_Finales[~df_Finales['Id_Carrera'].isin(listaDeCarreras)]
df_ALUCarrera = df_ALUCarrera[~df_ALUCarrera['Id_Carrera'].isin(listaDeCarreras)]
df_AluMateria = df_AluMateria[~df_AluMateria['Id_Carrera'].isin(listaDeCarreras)]

Alumnos repetidos

In [10]:
#Busco si hay datos repetidos en el df Alu carrera
df_ALUCarrera.nunique()

Id_Alumno           6025
Id_Carrera            13
Semestre_Ingreso       2
Año_Ingreso           10
Id_Ctro_Ext            6
Id_Modalidad           1
Fecha_Ingreso       1468
Activo                 2
Sexo                   2
Fecha_Nac           3665
dtype: int64

In [11]:
#Utilizo un gruopby para clasificar a los alumnos por sus ID y luego genero un Count de las carreras en las que estuvó.
AlumnosRepetidos = df_ALUCarrera.groupby(['Id_Alumno'])['Id_Carrera'].count().reset_index(name='Cantidad')
AlumnosRepetidos.sort_values(by='Cantidad',ascending=False)

,Id_Alumno,Cantidad
5107,0xD939FAAAF156527DFE6AEF66BD58EC08,3
2836,0x786298BEF8E9AA1F20C3D079463EAB73,2
3621,0x999E5707516A0B3CFA824CB027C30BD2,2
3646,0x9A9D99CFC91DD15E0C4DCA9A41E1EC35,2
4679,0xC5ED306DA9E24EE95F1A6A37830A36B3,2
...,...,...
2024,0x557709E67D117A17983F5217FBAA1173,1
2023,0x556C2E88464BADDB1645ED5B3AB047A0,1
2022,0x5552F249F93F26722CDF25018BDAC8B6,1
2021,0x554A7492AF4381123F4AE472EA9BEEB3,1


Columna Edad y Vacio de Columnas Irrelevantes

In [12]:
#Busco Vaciar todos los datos de la fecha de nacimiento que son un error ya que nadie menor a 17 podria estar cursando la carrera 
df_ALUCarrera.loc[(pd.DatetimeIndex(df_ALUCarrera['Fecha_Ingreso']).year - pd.DatetimeIndex(df_ALUCarrera['Fecha_Nac']).year )<17, 'Fecha_Nac'] = np.nan

In [13]:
#Creo una columna llamada edad 
df_ALUCarrera['Edad'] = pd.DatetimeIndex(df_ALUCarrera['Fecha_Ingreso']).year - pd.DatetimeIndex(df_ALUCarrera['Fecha_Nac']).year 

Estado del alumno en la materia

In [14]:
#Creo la columna y la lleno con un string arbitrario para poder modificarla luego
df_AluMateria['Condicion'] = 'Estado'

In [15]:
#Creo la condicion de promocionado.
df_AluMateria.loc[  (df_AluMateria['Nota_1p'] >= 7 ) & (df_AluMateria['Nota_1p'] != 11) 
                    & (df_AluMateria['Nota_2p'] >= 7 ) & (df_AluMateria['Nota_2p'] != 11)
                    & (df_AluMateria['Nota_TP'] >= 7 ) & (df_AluMateria['Nota_TP'] != 11)
                    , 'Condicion'] = 'Promocionado'
#Creo la condicion de Regular basandome en la de Promocionado y agregando algunas condiciones mas.
df_AluMateria.loc[  (df_AluMateria['Condicion'] !='Promocionado')
                    & ((df_AluMateria['Nota_TP'] >= 4) & (df_AluMateria['Nota_2p'] >= 4) & (df_AluMateria['Nota_1p'] >= 4) 
                        & (df_AluMateria['Nota_TP'] !=11) & (df_AluMateria['Nota_2p'] !=11) & (df_AluMateria['Nota_1p'] !=11)) 
                    | ((df_AluMateria['Nota_Recup']!= 11 ) & (df_AluMateria['Nota_Recup']>= 4 ))
                    , 'Condicion'] = "Regular"
#Creo la condicion de libre por descarte de la de Regular y Promocionado.
df_AluMateria.loc[  (df_AluMateria['Condicion'] != "Promocionado") & (df_AluMateria['Condicion'] !='Regular')
                    , 'Condicion'] = 'Libre'

Recursante

In [16]:
#Uso shift para usar el dato de la fila anterior y lo comparo con el de la fila actual.
#Si son iguales es porque el alumno esta recursando esa materia.
df_AluMateria['Tipo']=np.where(   ((df_AluMateria['Id_Materia'].shift() == df_AluMateria['Id_Materia']))
                                & ((df_AluMateria['Id_Carrera'].shift() == df_AluMateria['Id_Carrera']))
                                & ((df_AluMateria['Id_Alumno'].shift() == df_AluMateria['Id_Alumno']))
                                    ,'Recursante', np.nan)

Egresado, Cursante, Desertor

In [17]:
df_AluMateria['Egresado'] = 'Estado'

In [18]:
df_AluMateria = pd.merge(df_AluMateria,
                        df_Carreras[['Id_Carrera','Cuatrimestres']],
                        how='inner',
                        on=['Id_Carrera'])

In [19]:
pd.merge(df_AluMateria,
            df_ALUCarrera[['Id_Alumno','Año_Ingreso']],
            on=['Id_Alumno'])

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Fecha_Inscrip,Ctro_Ext,Id_Modalidad,Id_Comision,Nota_1p,...,Nota_TP,Fecha_TP,Nota_Recup,Fecha_Recup,Cuatrimestre,Condicion,Tipo,Egresado,Cuatrimestres,Año_Ingreso
0,0x003014EF4D7D79987F42DE8E3080AEFE,6,115,2,2006,2006-08-16 15:02:00,CO,P,7,11.0,...,11.0,2006-11-29 00:00:00,11.0,2006-11-27,4,Libre,nan,Estado,6,2004
1,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,1,2005,2005-03-10 18:12:00,CO,P,7,11.0,...,10.0,2005-12-05 15:44:00,11.0,2005-06-28,2,Libre,nan,Estado,6,2004
2,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,2,2005,2005-08-17 18:50:00,CO,P,7,11.0,...,10.0,2005-12-05 15:44:00,11.0,2005-11-21,2,Libre,Recursante,Estado,6,2004
3,0x003014EF4D7D79987F42DE8E3080AEFE,6,198,1,2005,2005-03-10 18:12:00,CO,P,7,7.0,...,10.0,2005-12-05 15:44:00,NaN,NaT,2,Promocionado,nan,Estado,6,2004
4,0x003014EF4D7D79987F42DE8E3080AEFE,6,374,1,2006,2006-03-06 12:33:00,CO,P,7,6.0,...,7.0,2006-07-07 00:00:00,NaN,NaT,3,Regular,nan,Estado,6,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134831,0xF870E768C0173041F6E7BC58DABADD4A,8,145,2,2001,2001-08-28 20:42:00,CO,P,7,11.0,...,10.0,2005-12-05 15:44:00,11.0,2001-11-22,2,Libre,nan,Estado,6,2001
134832,0xF870E768C0173041F6E7BC58DABADD4A,8,201,1,2001,2001-04-05 10:21:00,CO,P,7,11.0,...,10.0,2005-12-05 15:44:00,11.0,2001-07-03,1,Libre,nan,Estado,6,2001
134833,0xF870E768C0173041F6E7BC58DABADD4A,8,201,1,2003,2003-03-18 10:49:00,CO,P,1,11.0,...,10.0,2005-12-05 15:44:00,11.0,2003-06-26,1,Libre,Recursante,Estado,6,2001
134834,0xF870E768C0173041F6E7BC58DABADD4A,8,202,1,2001,2001-04-05 10:21:00,CO,P,7,11.0,...,10.0,2005-12-05 15:44:00,11.0,2001-06-28,1,Libre,nan,Estado,6,2001


In [20]:
df_AluMateria.loc[ (df_AluMateria['Cuatrimestre'] == df_AluMateria['Cuatrimestres'])
                    , 'Egresado'] = 'Egresado' 

Graficas

In [21]:
df_Cantidad_Alumnos_Año = df_ALUCarrera.groupby(['Id_Carrera','Año_Ingreso','Semestre_Ingreso'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_Cantidad_Alumnos_Año = pd.merge(df_Cantidad_Alumnos_Año,
                        df_Carreras[['Id_Carrera','Nombre_Corto']],
                        how='inner',
                        on=['Id_Carrera'])
df_Cantidad_Alumnos_Año
#hace un merge y trae el nombre del Id_Carrera 

,Id_Carrera,Año_Ingreso,Semestre_Ingreso,Cantidad,Nombre_Corto
0,1,2001,1,44,ANALISTA DE SISTEMAS
1,1,2001,2,5,ANALISTA DE SISTEMAS
2,1,2002,1,30,ANALISTA DE SISTEMAS
3,1,2002,2,12,ANALISTA DE SISTEMAS
4,1,2003,1,29,ANALISTA DE SISTEMAS
...,...,...,...,...,...
220,17,2007,2,1,GESTIÓN CONTABLE Y FINANCIERA
221,17,2008,1,12,GESTIÓN CONTABLE Y FINANCIERA
222,17,2009,1,24,GESTIÓN CONTABLE Y FINANCIERA
223,17,2010,1,12,GESTIÓN CONTABLE Y FINANCIERA


In [22]:
px.line(df_Cantidad_Alumnos_Año, x="Año_Ingreso",y='Cantidad', color="Nombre_Corto", title='Cantidad de alumnos x año y semestre',animation_frame= 'Semestre_Ingreso')

In [23]:
px.histogram(df_ALUCarrera, 
            x="Edad", 
            title='Edad de alumnos',
            color='Sexo',
            animation_frame=df_ALUCarrera['Año_Ingreso'].sort_values())

In [24]:
df_Sexo_Alumno_xCarrera = df_ALUCarrera.groupby(['Id_Carrera','Sexo'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_Sexo_Alumno_xCarrera = pd.merge(df_Sexo_Alumno_xCarrera,
                        df_Carreras[['Id_Carrera','Nombre_Corto']],
                        how='inner',
                        on=['Id_Carrera'])
df_Sexo_Alumno_xCarrera

,Id_Carrera,Sexo,Cantidad,Nombre_Corto
0,1,F,92,ANALISTA DE SISTEMAS
1,1,M,490,ANALISTA DE SISTEMAS
2,2,F,233,MARKETING
3,2,M,408,MARKETING
4,3,F,218,PUBLICIDAD
5,3,M,288,PUBLICIDAD
6,4,F,224,ADMINISTRACIÓN DE EMPRESAS
7,4,M,392,ADMINISTRACIÓN DE EMPRESAS
8,5,F,505,DISEÑO GRÁFICO
9,5,M,458,DISEÑO GRÁFICO


In [25]:
px.bar(df_Sexo_Alumno_xCarrera, x='Nombre_Corto',y='Cantidad', color='Sexo')

In [31]:
df_AluMateria

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Fecha_Inscrip,Ctro_Ext,Id_Modalidad,Id_Comision,Nota_1p,...,Fecha_2p,Nota_TP,Fecha_TP,Nota_Recup,Fecha_Recup,Cuatrimestre,Condicion,Tipo,Egresado,Cuatrimestres
0,0x003014EF4D7D79987F42DE8E3080AEFE,6,115,2,2006,2006-08-16 15:02:00,CO,P,7,11.0,...,2006-11-20,11.0,2006-11-29 00:00:00,11.0,2006-11-27,4,Libre,nan,Estado,6
1,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,1,2005,2005-03-10 18:12:00,CO,P,7,11.0,...,2005-06-22,10.0,2005-12-05 15:44:00,11.0,2005-06-28,2,Libre,nan,Estado,6
2,0x003014EF4D7D79987F42DE8E3080AEFE,6,121,2,2005,2005-08-17 18:50:00,CO,P,7,11.0,...,2005-11-14,10.0,2005-12-05 15:44:00,11.0,2005-11-21,2,Libre,Recursante,Estado,6
3,0x003014EF4D7D79987F42DE8E3080AEFE,6,198,1,2005,2005-03-10 18:12:00,CO,P,7,7.0,...,2005-06-23,10.0,2005-12-05 15:44:00,NaN,NaT,2,Promocionado,nan,Estado,6
4,0x003014EF4D7D79987F42DE8E3080AEFE,6,374,1,2006,2006-03-06 12:33:00,CO,P,7,6.0,...,2006-06-26,7.0,2006-07-07 00:00:00,NaN,NaT,3,Regular,nan,Estado,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130668,0xF870E768C0173041F6E7BC58DABADD4A,8,145,2,2001,2001-08-28 20:42:00,CO,P,7,11.0,...,2001-11-05,10.0,2005-12-05 15:44:00,11.0,2001-11-22,2,Libre,nan,Estado,6
130669,0xF870E768C0173041F6E7BC58DABADD4A,8,201,1,2001,2001-04-05 10:21:00,CO,P,7,11.0,...,2001-06-21,10.0,2005-12-05 15:44:00,11.0,2001-07-03,1,Libre,nan,Estado,6
130670,0xF870E768C0173041F6E7BC58DABADD4A,8,201,1,2003,2003-03-18 10:49:00,CO,P,1,11.0,...,2003-06-12,10.0,2005-12-05 15:44:00,11.0,2003-06-26,1,Libre,Recursante,Estado,6
130671,0xF870E768C0173041F6E7BC58DABADD4A,8,202,1,2001,2001-04-05 10:21:00,CO,P,7,11.0,...,2001-06-14,10.0,2005-12-05 15:44:00,11.0,2001-06-28,1,Libre,nan,Estado,6


In [27]:
# Cantidad de alumnos por materia a lo largo de los años
# Nota promedio por carrera a lo largo de los años en los finales.
# Nota promedio por carrera dividida en hombres y mujeres